# CS 558 Homework - Due November 7th
## Saoirse Mooney & Joshua Meharg
I pledge my honor that I have abided by the Stevens Honor System.


Implement the Harris corner detector for a gray value image. You should leverage 
the code developed so far in the course and compute the “cornerness” response 
function of each point based on the second moment matrix. The final feature 
selection will then pick the 1000 strongest points as the features of the image. 
Execute this detector on greyscale versions of the two images in the 
TwoViewAlignment  data subdirectory. Include the results in your report. 

Import statements

In [78]:
%matplotlib inline 
from matplotlib import pyplot as plt
import numpy as np
import random as rand
import math
from tqdm import tqdm
import copy 
from PIL import Image as im
from IPython.display import Image
import cv2
from numpy.core.memmap import uint8
import imutils

Convery both images to grey scale and open side-by-side

In [79]:
def sidebyside(img1in, img2in):
  """
  Summary: Take two images and creates a new image of them side-by-side
  """
  #creates new image with img1in on the left and img2in on the right
  print("img1 shape = " + str(img1in.shape))
  print("img2 shape = " + str(img2in.shape))

  img1 = copy.copy(img1in)
  row1 = len(img1)
  col1 = len(img1[0])

  img2 = copy.copy(img2in)
  row2 = len(img2)
  col2 = len(img2[0])
  #(rows, columns)
  row_count = max(row1, row2)
  result = np.zeros([row_count, col1+col2])
  print("result shape = " + str(result.shape))
   #painting img1
  for i in range(0, row1):
    for j in range(0, col1):
      result[i][j] = img1[i][j]

  #painting img2
  for i in range(0, row2):
    for j in range(0, col2):
      result[i][j+col1] = img2[i][j]

  return result  

In [80]:
def generateWindow(s):
    X = np.zeros([s,s], dtype=int)
    Y = np.zeros([s,s], dtype=int)
    rnge = math.floor(s/2)
    curr_r = -1*rnge
    for i in range(0,s):
        Y[i] = np.full(s, curr_r)        
        for j in range(0,s):
            X[j][i] = curr_r
        curr_r += 1 
    
    return X,Y

In [81]:
def generateGaussianFilter(sigma): 
    # generate windows
    X,Y = generateWindow(6*sigma-1)

    coef = 1 / (2 * math.pi * sigma**2)
    s = len(X)
    result = np.zeros([s, s])
    
    for i in range(0, s):
        for j in range(0, s):
            xi = X[i][j]
            yi = Y[i][j]
            power = -1 * (xi*xi + yi*yi) / (2*sigma*sigma)
            result[i][j] = coef * math.exp(power)
    
    filter_sum = np.sum(np.sum(result))
    print("The sum of the filter coefficients is ",filter_sum)
    if filter_sum < 0.95:
        print("WARNING: the filter coefficients do not sum to 1")
    return result

In [82]:
def applyGaussian(filtr, input_img):
    s = len(filtr)
    rows = len(input_img)-(2*math.floor(s/2))
    cols = len(input_img[0])-(2*math.floor(s/2))
    result = np.zeros([rows, cols])
    
    for x in tqdm(range(0, rows)):
        for y in range(0, cols):
            local_pixels = np.zeros([s, s])
            # fill temp matrix w elems from image
            for i in range(0, s):
                for j in range(0, s):
                    local_pixels[i][j] = input_img[x+i][y+j]
            # convolve filter and local pixels   
            result[x][y]=np.sum(np.sum(np.multiply(filtr,local_pixels)))
            #result[x][y] = np.sum(np.multiply(local_pixels.flatten(), np.flip(filtr.flatten())))
        
    return result             

In [83]:
def convolve(filtr, img):
    image_rows = len(img)
    image_columns = len(img[0])
    # create a copy of image for padding
    s = len(filtr)
    print("img size",np.shape(img))

    result = np.zeros([image_rows, image_columns])
    
    for x in tqdm(range(0,image_rows-2)):
        for y in range(0,image_columns-2):
            local_pixels = np.zeros([s, s])
            # fill temp matrix w elems from image
            for i in range(0, s):
                for j in range(0, s):
                    local_pixels[i][j] = img[x+i][y+j]
            result[x][y]=np.sum(np.sum(np.multiply(filtr,local_pixels)))
            #result[x][y]=np.sum(np.multiply(local_pixels.flatten(), np.flip(filtr.flatten())))
                        
    return result

In [84]:
def gradient_computation(Gx, Gy):
    rows = len(Gx)
    columns = len(Gx[0])
    result = np.zeros([rows, columns])
    
    for i in range(0, rows):
        for j in range(0, columns):
            gx = Gx[i][j]
            gy = Gy[i][j]
            G = math.sqrt((gx**2) + (gy**2))
            result[i][j] = G
    return result
    

In [85]:
def non_max_sup(imgin):
  img = copy.copy(imgin)
  row = len(img)
  col = len(img[0])
    
  
  for i in tqdm(range(0, row)):
    #print("i = " + str(i))
    for j in range(0, col):
      #on upper left edge
      if (i == 0 and j==0):
        if (img[i][j] <= img[i][j+1]
            or img[i][j] <= img[i+1][j]
            or img[i][j] <= img[i+1][j+1]):
              img[i][j] = 0
      
      #on bottom left
      elif (i == len(img)-1 and j==0):
        if (img[i][j] <= img[i][j+1]
            or img[i][j] <= img[i-1][j]
            or img[i][j] <= img[i-1][j+1]):
              img[i][j] = 0
      
      #on top right
      elif (i == 0 and j==len(img[0])-1):
        if (img[i][j] <= img[i][j-1]
            or img[i][j] <= img[i+1][j]
            or img[i][j] <= img[i+1][j-1]):
              img[i][j] = 0

      #on bottom right
      elif (i== len(img)-1 and j == len(img[0])-1):
        if (img[i][j] <= img[i][j-1]
            or img[i][j] <= img[i-1][j]
            or img[i][j] <= img[i-1][j-1]):
              img[i][j] = 0

      #on left edge
      elif (j == 0):
        if (img[i][j] <= img[i-1][j] 
            or img[i][j] <= img[i-1][j+1]
            or img[i][j] <= img[i][j+1]
            or img[i][j] <= img[i+1][j+1]
            or img[i][j] <= img[i+1][j]):
            #dont keep current point
              img[i][j] = 0
      
      #on right edge
      elif (j == len(img[0])-1):
        if (img[i][j] <= img[i-1][j] #above
            or img[i][j] <= img[i-1][j-1] #top left
            or img[i][j] <= img[i][j-1] #left
            or img[i][j] <= img[i+1][j-1] #bottom left
            or img[i][j] <= img[i+1][j]):
            #dont keep current point
              img[i][j] = 0
            
      #on top edge
      elif (i == 0):
        if (img[i][j] <= img[i][j-1] #left
            or img[i][j] <= img[i+1][j-1] #bottom left
            or img[i][j] <= img[i+1][j] #below
            or img[i][j] <= img[i+1][j+1] #bottom right
            or img[i][j] <= img[i][j+1]):
            #dont keep current point
              img[i][j] = 0
            
      #on bottom edge
      elif (i == len(img)-1):
        if (img[i][j] <= img[i][j-1] #left
            or img[i][j] <= img[i-1][j-1] #top left
            or img[i][j] <= img[i-1][j] #above
            or img[i][j] <= img[i-1][j+1] #top right
            or img[i][j] <= img[i][j+1]):
            #dont keep current point
              img[i][j] = 0

      # middle case
      elif (img[i][j] <= img[i-1][j-1] 
          or img[i][j] <= img[i][j-1] 
          or img[i][j] <= img[i+1][j-1]
          or img[i][j] <= img[i+1][j]
          or img[i][j] <= img[i+1][j+1]
          or img[i][j] <= img[i][j+1]
          or img[i][j] <= img[i-1][j+1]
          or img[i][j] <= img[i-1][j]):
          #dont keep current point
            img[i][j] = 0
  return img

In [86]:
# pick top 1000 feature points before applying non max suppression
def top1000(imgin):
  img = copy.copy(imgin)
  lst = []
  row = len(img)
  col = len(img[0])
  for i in tqdm(range (0, 1000)):
    index = np.unravel_index(np.argmax(img), img.shape)
    lst += [index]
    img[index] = 0
  return lst

In [87]:
#outputs an image result with shape of img with white pixels for coordinates in lst
#if u specify white as true than paints with 255
def paint(img, lst, white=False):
  image_rows = len(img)
  image_columns = len(img[0])
  result = np.zeros([image_rows, image_columns])
  for i in range(0, len(lst)):
    if (white == False):
      result[lst[i]] = img[lst[i]]
    elif (white):
      result[lst[i]] = 255
  return result


In [88]:
def process(imgin):
  """
  Summary: #main function for applying guassian, creating second moment matrix, taking determinate, 
  getting 1000 feature points and applying non-max suppression
  """

  horizontal_sobel_filter = np.array([[1, 0, -1], [2, 0, -2], [1, 0, -1]])
  vertical_sobel_filter = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]])

  gaussian_filtrp = generateGaussianFilter(1)
  gauss_img_matrixp = applyGaussian(gaussian_filtrp, imgin)

  # gaussian already applied to input img
  XXp = convolve(horizontal_sobel_filter, (convolve(horizontal_sobel_filter, gauss_img_matrixp))) #caluclate Ixx
  XYp = convolve(vertical_sobel_filter, (convolve(horizontal_sobel_filter, gauss_img_matrixp))) #calculate Ixy
  YYp = convolve(vertical_sobel_filter, (convolve(vertical_sobel_filter, gauss_img_matrixp))) #calculate Iyy

  det_xx_yyp = np.multiply(XXp, YYp)
  det_xy_xyp = np.multiply(-1,np.multiply(XYp,XYp))

  #determinate
  res_imgp = np.sum([det_xx_yyp, det_xy_xyp], axis=0)

  
  features = top1000(res_imgp)
  features1 = paint(res_imgp, features)
  features2 = non_max_sup(features1)

  # fe1 = im.fromarray(features1.astype(uint8))
  # fe1.save('initial_top1000_features.jpg')
  # fe2 = im.fromarray(features2.astype(uint8))
  # fe2.save('feature_points_afternon-max.jpg')

  return features1, features2


In [89]:
def featurepoints(img):
  #returns a list of tuples representing coordinates of feature points of img
  lst = []
  row = len(img)
  col = len(img[0])
  for i in tqdm(range(0, row)):
    for j in range(0, col):
      if (img[i][j] > 0):
        lst += [(i, j)]
  return lst

In [90]:
def offset(l1, imgin):
  out = l1
  cols = len(imgin[0])
  # offsets the col value inorder to print line correctly for side by side img
  for i in range(0, len(l1)):
    out[i] = (l1[i][0], l1[i][1] + cols)
  return out

In [91]:
def secondmax(imgin):
  img = copy.copy(imgin)
  row = len(img)
  col = len(img[0])
  index = np.unravel_index(np.argmax(img), img.shape)
  img[index] = 0
  return 

In [92]:
def getmax(imgin):
  return np.unravel_index(np.argmax(imgin), imgin.shape)

In [93]:

def paintlines(imgin, c1, c2):
  img = copy.copy(imgin)
  # c1 is the feature points coords for the left image and c2 for the right
  for i in range(0, len(c1)):
    img = cv2.line(img, [c1[i][1], c1[i][0]], [c2[i][1], c2[i][0]], (255, 87, 51), 2) 
    # img = cv2.circle(img, [c1[i][1], c1[i][0]], 6, (255, 0, 0), 1)
    # img = cv2.circle(img, [c2[i][1], c2[i][0]], 6, (255, 0, 0), 1)
    # break
  return img

In [94]:
def getbox(coord, n, img):
    #coord in format (x, y)
    #returns list of window of values with coord at center that is
    #3^n by 3^n
    out = []
    topleftcorner = coord
    for i in range(0, n):
        topleftcorner = (topleftcorner[0]-1, topleftcorner[1]-1)
    
    curr = topleftcorner
    for i in range(0, 2*n+1):
        for j in range(0,  2*n+1):
            out += [img[curr[0]+i][curr[1]+j]]
    return out

In [95]:
def NCCpoint(list1, list2, img1, img2, n):
  # does NCC with n x n area window
  if (n%2==0 or n==1):
    print("error, n has to be odd and greater than 1")
  n = int((n)/2)
  out = []
  # Takes in feature points of img2 and img2 which is stored as list2 and list2 and calculates the SSD distance between two sets of feature points 
  for i in (tqdm(range(0, len(list1)))):
    for j in range(0, len(list2)):
        # window around the center point for feature
        # point from img2
        center1 = img1[list1[i]]
        box1 = getbox(list1[i], n, img1)
        mean1 = (sum(box1)) / pow(2*n+1, 2)

        # window around the center point for feature
        # point from img2
        
        center2 = img2[list2[j]]
        box2 = getbox(list2[j], n, img2)
        mean2 = (sum(box2)) / pow(2*n+1, 2)

        # I1(x) - I1mean * I2(x) - I2mean
        top = 0
        for t in range(0, len(box1)):
            top += (box1[t] - mean1)*(box2[t] - mean2)
        
        #(I1(x) - I1mean)^2
        botl = 0
        for t in range(0, len(box1)):
            botl += pow((box1[t]-mean1),2)
        
        #(I2(x) - I2mean)^2
        botr = 0
        for t in range(0, len(box1)):
            botr += pow((box2[t]-mean2),2)

        currans = top / math.sqrt((botr*botl))

        out += [(list1[i], list2[j], currans)]
  return out

In [96]:
def max20(listin):
  copy = listin
  out1 = []
  out2 = []
  max = 0
  for x in tqdm(range(0, 20)):
    for i in range(0, len(listin)):
      if (copy[i][2] > max):
        max = copy[i][2]
        max_in = i

    out1 += [listin[max_in][0]]
    out2 += [listin[max_in][1]]
    copy[max_in] = (copy[max_in][0], copy[max_in][1], -1)
    max = 0
  return out1, out2


## Discussion:
SSD takes a template image and iterates over its discrete pixels to find where those pixels are located in the corresponding image. It does this by taking the sum of squared differences by calculating the residual error at every displacement in the test image. it assumes taht corresponding pixel values between the template and the test are the same. NNC on the other hand find displacements where the cross-correlation between the images is maximized. NNC works better for images with different exposures and the range will always be [-1,1].

In our test, NCC worked well in matching similar points which are mainly in the middle of the side by side image. There appear to be around 2 outliers, but the bulk of the points, the other 18, seem to match well

## Extra Credit
When we rotate one of the images by 45 degrees as you can see in image problem1_extracredit the features match less efficiently. This is we use the windows around and including each feature point to compare and if one image has rotated, the values in the window will change even if the central pixels are the same. This can be solved by matching the rotations of the two images/correcting the planes of the image so that they are equivalent to properly extract the equivalent windows


In [131]:
def problem1_main(img1, img2, rotate=False, rotate_img=0,rotate_deg=0):
  """
  Summary: Takes in two images and rotation wants, and performs corner detection, non-maxima suppression, and feature detection to compute the SSD od the images.
  Parameters:
  img1 (str): jpg
  img2 (Str): jpg
  rotate (bool): True if you want one of the images to be rotates
  rotate_img (int): Specify img1 as 1 or img2 as 2
  rotate_deg (int): degree to rotate specified image

  Return:
  Image of correspondances
  """
  # open images and convert to grey scale
  im1 = im.open(img1).convert("L")
  im1 = np.array(im1)
  im2 = im.open(img2).convert("L")
  im2 = np.array(im2)

  if rotate==True:
    if rotate_img == 1:
      im1 = imutils.rotate_bound(im1,rotate_deg)
    elif rotate_img == 2:
      im2 = imutils.rotate_bound(im2,rotate_deg)
    else:
      print("Error: rotate_img not valid")
      return
  c_1, p_im1 = process(im1)
  c_2, p_im2 = process(im2)

  if rotate!=True:
    c_1 = im.fromarray(c_1.astype(uint8))
    c_1.save('uttowerleft_top100features.jpg')

    c_2 = im.fromarray(c_2.astype(uint8))
    c_2.save('uttowerright_top100features.jpg')

    pp_im2 = im.fromarray(p_im2.astype(uint8))
    pp_im2.save('uttowerright_features_after_non-max.jpg')

    pp_im1 = im.fromarray(p_im1.astype(uint8))
    pp_im1.save('uttowerleft_features_after_non-max.jpg')
  else:
    c_1 = im.fromarray(c_1.astype(uint8))
    c_1.save('uttowerleft_top100features_rotated.jpg')

    c_2 = im.fromarray(c_2.astype(uint8))
    c_2.save('uttowerright_top100features.jpg')

    pp_im2 = im.fromarray(p_im2.astype(uint8))
    pp_im2.save('uttowerright_features_after_non-max.jpg')

    pp_im1 = im.fromarray(p_im1.astype(uint8))
    pp_im1.save('uttowerleft_features_after_non-max_rotated.jpg')


  fp1 = featurepoints(p_im1)
  fp2 = featurepoints(p_im2)
  NCC = NCCpoint(fp1, fp2, im1, im2, 9)
  points1, points2 = max20(NCC)
  img_canvas = sidebyside(im1, im2)
  ans_img = paintlines(img_canvas, points1, offset(points2, im2))
  ans_img = im.fromarray(ans_img.astype(np.uint8))
  if rotate != True:
    ans_img.save('problem1_ans.jpg')
  else:
    ans_img.save('problem1_extracredit.jpg')

  return Image('problem1_ans.jpg'), points1, points2, NCC


In [133]:
t, points1, points2, ohno = problem1_main('uttower_left.jpg', 'uttower_right.jpg')
#ta, points1a, points2a, dontworry = problem1_main('uttower_left.jpg', 'uttower_right.jpg', True, 1, 45)

The sum of the filter coefficients is  0.9818147610543744


100%|██████████| 679/679 [00:23<00:00, 29.08it/s]


img size (679, 1020)


100%|██████████| 677/677 [00:15<00:00, 43.30it/s]


img size (679, 1020)


...

img size (679, 1020)


100%|██████████| 677/677 [00:15<00:00, 43.10it/s]


img size (679, 1020)


100%|██████████| 20/20 [00:00<00:00, 54.20it/s]


img1 shape = (683, 1024)
img2 shape = (683, 1024)
result shape = (683, 2048)


In [141]:
matching_points = []
im1 = im.open('uttower_left.jpg').convert("L")
im1 = np.array(im1)
points3 = points2

for i in range(0, len(points3)):
  points3[i] = (points3[i][0], points3[i][1] - len(im1[0]))

for i in range(0, len(points1)):
  matching_points += [(points1[i], points3[i])]
matching_points

[((549, 901), (518, 439)),
 ((557, 779), (524, 322)),
 ((535, 775), (502, 319)),
 ((495, 694), (460, 242)),
 ((371, 894), (347, 444)),
 ((552, 600), (516, 173)),
 ((371, 894), (357, 531)),
 ((552, 592), (515, 148)),
 ((552, 600), (515, 122)),
 ((579, 62), (524, 322)),
 ((552, 600), (515, 148)),
 ((552, 584), (516, 173)),
 ((552, 584), (515, 148)),
 ((552, 608), (515, 156)),
 ((552, 600), (515, 156)),
 ((552, 592), (516, 173)),
 ((513, 444), (353, 489)),
 ((541, 581), (504, 153)),
 ((553, 624), (516, 173)),
 ((552, 592), (515, 122))]

# Problem 2: Two View Image Alignment

Using the feature detection code in the problem 1, build a set of putative 
feature correspondences using the following rules: a1) Selected the top 20 
features based on the similarity measure score; a2) Select 30 random 
correspondences

In [142]:
# Top 20 features as used in problem 1
print(matching_points)

[((549, 901), (518, 439)), ((557, 779), (524, 322)), ((535, 775), (502, 319)), ((495, 694), (460, 242)), ((371, 894), (347, 444)), ((552, 600), (516, 173)), ((371, 894), (357, 531)), ((552, 592), (515, 148)), ((552, 600), (515, 122)), ((579, 62), (524, 322)), ((552, 600), (515, 148)), ((552, 584), (516, 173)), ((552, 584), (515, 148)), ((552, 608), (515, 156)), ((552, 600), (515, 156)), ((552, 592), (516, 173)), ((513, 444), (353, 489)), ((541, 581), (504, 153)), ((553, 624), (516, 173)), ((552, 592), (515, 122))]


In [136]:
# 30 random correspondences
rand_t = rand.sample(ohno,30)
rand_1 = []
rand_2 = []
rand_l = []
for i in range(0, len(rand_t)):
  rand_1.append(rand_t[i][0])
  rand_2.append(rand_t[i][1])
  rand_l.append((rand_t[i][0],rand_t[i][1]))


Implement a RANSAC-based method to estimate an affine transformation 
between uttower_left.jpg and uttower_right.jpg in the TwoViewAlignment 
directory. First, run using as input only the putative correspondences defined in a1. Second use the aggregate putative correspondences of BOTH a1 AND 
a2. Show the inlier matches determined by RANSAC procedure (with 
adaptive iteration number) and compute their average feature reprojection 
error between the images.  Discuss what is the expected number of RANSAC 
iterations for each experiment and what is the actual number observed in 
practice. Include the results in your report. 

## Discussion: 
RANSAC will remove the outlier correspondances and leave only the inliners as long as the inliners outnumber the outliners

The expected number of RANSAC iterations must be enough to ensure that four correct correspondances are selected from the list of all correspondances. If we are selecting the top 20 most likely correspondances, then the number of iterations will be much lower to reach an ideal homography. With a random selection of 30 correspondances, it is possible that the number of correct matches is less than the number of outliers, which would render the hoography unsolvable.

In our code, neither selection of points works to generate a homography that causes the matches matrix multiplied by the homography to be close to 0, indicating a good model. This is likely due to an incorrect implementation of RANSAC for an affine transformation.

In [123]:
def affine_ransac(coor, N, e):
  """
  Summary: randomly samples 4 coorepondances and builds a homography. repeats N times.
  Parameters:
  coor (list(((int, int)), ((int, int))): coordinates and value
  N (int): number of iterations
  e (int): accepted error
  Returns:
  model (int, array): number of inliners and the corresponding homography
  """
  # randomly sample s correspondances -- s=4
  # hypothesize the model h -- find the smallest eigen of At * a
  # plug each point from the correspondances into the model and see if the error is small enough
    # plug in one corr to the model and see if you get the correct matching coor -- minimize the error between the output and the correct coor
  #coor = [((xs, ys),vs), ((xd,yd), vd)]
  models = []
  while N>0:
    num_correspondances = len(coor)
    rand_is = np.random.choice(a=num_correspondances, size=4, replace=False) # random indicies -- get 4 samples
    s0 = coor[rand_is[0]]
    c0_s = s0[0]
    c0_d = s0[1]
    s1 = coor[rand_is[1]]
    c1_s = s1[0]
    c1_d = s1[1]
    s2 = coor[rand_is[2]]
    c2_s = s2[0]
    c2_d = s2[1]
    s3 = coor[rand_is[3]]
    c3_s = s3[0]
    c3_d = s3[1]

    s_arr = [s0,s1,s2,s3]  
    #print(s_arr)  

    # construct A matrix with 8 degrees of freedom
    A = np.zeros((8,9))
    A[0] = [c0_s[0], c0_s[1],1, 0,0,0, -1*c0_d[0]*c0_s[0], -1*c0_d[0]*c0_s[1], -1*c0_d[0]]
    A[1] = [0,0,0, c0_s[0],c0_s[1],1, -1*c0_d[1]*c0_s[0],-1*c0_d[1]*c0_s[1],-1*c0_d[1]]

    A[2] =  [c1_s[0], c1_s[1],1, 0,0,0, -1*c1_d[0]*c1_s[0], -1*c1_d[0]*c1_s[1], -1*c1_d[0]]
    A[3] = [0,0,0, c1_s[0],c1_s[1],1, -1*c1_d[1]*c1_s[0],-1*c1_d[1]*c1_s[1],-1*c1_d[1]]

    A[4] = [c2_s[0], c2_s[1],1, 0,0,0, -1*c2_d[0]*c2_s[0], -1*c2_d[0]*c2_s[1], -1*c2_d[0]]
    A[5] = [0,0,0, c2_s[0],c2_s[1],1, -1*c2_d[1]*c2_s[0],-1*c2_d[1]*c2_s[1],-1*c2_d[1]]

    A[6] =  [c3_s[0], c3_s[1],1, 0,0,0, -1*c3_d[0]*c3_s[0], -1*c3_d[0]*c3_s[1], -1*c3_d[0]]
    A[7] = [0,0,0, c3_s[0],c3_s[1],1, -1*c3_d[1]*c3_s[0],-1*c3_d[1]*c3_s[1],-1*c3_d[1]]

    #print("A",A)
    A_t = np.transpose(A)
    #print("At", A_t)
    eig_val, eig_vec = np.linalg.eig(np.matmul(A_t,A))
    lam_i = np.argmin(eig_val)
    lam = eig_val[lam_i]
    h = eig_vec[lam_i]
    h = h.reshape((3,3)) # eigvec h with smallest eigval minimizes the loss function

    # have h as out hypothesized homography -- now test
    
    #h, m = cv2.findHomography(np.array(p1), np.array(p2), cv2.RANSAC)
    for c in range(0, num_correspondances):
      inliners = np.array([])
      xs = coor[c][0][0]
      ys = coor[c][0][1]
      xd = coor[c][1][0]
      yd = coor[c][1][1]
      src = np.array([[xs],[ys],[1]])
      dst = np.matmul(h, src)
      h_xd = dst[0] / dst[2]
      h_yd = dst[1] / dst[2]
      if abs(h_xd - xd)<=e and abs(h_yd - yd)<=e:
        inliners = np.append(inliners, coor[c])

    models.append((len(inliners), h))

    N-=1

  # return model w largest number of inliners
  models.sort(key=lambda tup: tup[0])
  return models[-1]


In [124]:
h = affine_ransac(matching_points, 20, 100)
h

(0, array([[-1.15018352e-03, -1.11516771e-03, -4.87378677e-04],
        [-6.48710661e-02,  6.56142576e-01, -7.45567213e-01],
        [ 9.68085862e-02, -4.26652917e-03,  2.31579056e-03]]))

In [137]:
rand_h = affine_ransac(rand_l, 50, 100)

Warp one image onto the other using the affine transformation estimates. To 
do this, you will need to learn about MATLAB maketform and imtransform 
functions (or their python/C counterparts). Create a new image big enough 
to hold the panorama and composite the two images into it. You may 
composite by simply averaging the pixel values where the two images 
overlap. Include the results in your report. 

In [138]:
def warp(img1,img2, h):
  """
  Summary: Warps one image onto another using affine transformation estimates
  """
  h1,w1 = img1.shape
  pts = np.float32([[0,0], [0,h1-1], [w1-1,h1-1], [w1-1,0]])
  dst = cv2.perspectiveTransform(pts, h)
  res = cv2.polylines(img2, [np.int32(dst)], True, 255, 3, cv2.LINE_AA)
  return res

Because our homography had errors we can stitch the images together using the builtin cv2 functions

In [139]:

paths = ['uttower_left.jpg','uttower_right.jpg']
imgs = []
for image in paths:
    img = cv2.imread(image)
    imgs.append(img)

stitcher = cv2.Stitcher_create()
err, stitched = stitcher.stitch(imgs)
if (not err == True):
    cv2.imwrite("stitched_img.jpg", stitched)
